# Balanceo de dataset combinando SMOTE + Tomek

In [1]:
import pandas as pd

df = pd.read_csv('../data/TA1C_dataset_detection_train_cleaned.csv')

In [2]:
# Convertir etiquetas de texto a valores numéricos
label_mapping = {'Clickbait': 1, 'No': 0}
df['Tag Value'] = df['Tag Value'].map(label_mapping)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Probar diferentes configuraciones de n-gramas y representaciones
configurations = [
    {"ngram_range": (1, 1), "representation": "tfidf"},  # Unigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "tfidf"},  # Unigramas + Bigramas con TF-IDF
    {"ngram_range": (1, 3), "representation": "tfidf"},  # Unigramas + Bigramas + Trigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "binary"},  # Unigramas + Bigramas con representación binaria
    {"ngram_range": (1, 2), "representation": "frequency"},  # Unigramas + Bigramas con frecuencia
]

for config in configurations:
    print(f"Configuración: ngram_range={config['ngram_range']}, representación={config['representation']}")
    
    # Configurar el vectorizador
    if config["representation"] == "tfidf":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    elif config["representation"] == "binary":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], binary=True)
    elif config["representation"] == "frequency":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], use_idf=False)
    
    # Crear representaciones para el conjunto de entrenamiento y desarrollo
    X = vectorizer.fit_transform(df['cleaned_lemmatized_text'])
    y = df['Tag Value']
    
    # Verificar las dimensiones
    print(f"Dimensiones de X: {X.shape}")

Configuración: ngram_range=(1, 1), representación=tfidf
Dimensiones de X: (2800, 10591)
Configuración: ngram_range=(1, 2), representación=tfidf
Dimensiones de X: (2800, 47819)
Configuración: ngram_range=(1, 3), representación=tfidf
Dimensiones de X: (2800, 98988)
Configuración: ngram_range=(1, 2), representación=binary
Dimensiones de X: (2800, 47819)
Configuración: ngram_range=(1, 2), representación=frequency
Dimensiones de X: (2800, 47819)


## Dividir dataset en entrenamiento y desarrollo

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_dev, y_train, y_dev =train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=0,
    stratify=y,
    shuffle=True
)



## Balancear el conjunto de entrenamiento

In [5]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)

In [6]:
from sklearn.decomposition import TruncatedSVD

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0)  # Reducir a 100 dimensiones
X_train_reduced = svd.fit_transform(X_resampled)
X_dev_reduced = svd.transform(X_dev)

# Verificar las dimensiones después de la reducción
print(f"Dimensiones reducidas de X_train: {X_train_reduced.shape}")
print(f"Dimensiones reducidas de X_dev: {X_dev_reduced.shape}")

Dimensiones reducidas de X_train: (3002, 100)
Dimensiones reducidas de X_dev: (700, 100)


## Entrenar usando random forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
rf = RandomForestClassifier(random_state=0)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    rf,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train_reduced, y_resampled)

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.8s
[CV] END max_de

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de Random Forest
rf_model = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=5,n_estimators=200, random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
rf_scores = cross_val_score(rf_model, X_train_reduced, y_resampled, cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Random Forest:")
print(f"F1-macro scores por fold: {rf_scores}")
print(f"F1-macro promedio: {rf_scores.mean():.4f}")

Random Forest:
F1-macro scores por fold: [0.81357154 0.80187846 0.92996187 0.91480097 0.90654102]
F1-macro promedio: 0.8734


In [15]:
from sklearn.metrics import classification_report, confusion_matrix

# Entrenar el modelo en todo el conjunto de entrenamiento
rf_model.fit(X_train_reduced, y_resampled)

# Predecir sobre el conjunto de validación/desarrollo
y_pred = rf_model.predict(X_dev_reduced)
y_true = y_dev

# Imprimir el reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_true, y_pred, digits=4))

# Imprimir la matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_true, y_pred))

Reporte de clasificación:
              precision    recall  f1-score   support

           0     0.7574    0.9661    0.8491       501
           1     0.7213    0.2211    0.3385       199

    accuracy                         0.7543       700
   macro avg     0.7394    0.5936    0.5938       700
weighted avg     0.7472    0.7543    0.7039       700

Matriz de confusión:
[[484  17]
 [155  44]]


## Usando MLP

### Pruebas usando GridSearch

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
mlp = MLPClassifier(random_state=0, max_iter=2000)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (150,)],
    'alpha': [0.0001, 0.001, 0.01],
    'solver': ['adam', 'lbfgs'],
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    mlp,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train_reduced, y_resampled)

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=   7.3s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=   8.3s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=   8.2s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   3.2s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   3.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  13.0s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   2.4s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=   8.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   4.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   5.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), solver=adam; total time=   7.5s
[CV] END alpha=0.0001, hidden_layer_sizes

/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  16.2s
[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  13.7s
[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  13.3s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  16.7s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=   3.3s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=   4.2s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=   7.4s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=   8.8s
[CV] END .alpha=0.01, hidden_layer_sizes=(100,), solver=adam; total time=   9.0s


...

[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  35.5s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=   5.0s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=   8.5s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=   7.2s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=adam; total time=   8.2s
[CV] END alpha=0.01, hidden_layer_sizes=(100,), solver=lbfgs; total time=  32.0s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  25.2s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  26.2s
[CV] END .alpha=0.01, hidden_layer_sizes=(150,), solver=adam; total time=   4.3s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  25.6s
[CV] END .alpha=0.01, hidden_layer_sizes=(150,), solver=adam; total time=   7.6s
[CV] END alpha=0.01, hidden_layer_sizes=(100, 50), solver=lbfgs; total time=  39.8s
[CV] END

/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END alpha=0.01, hidden_layer_sizes=(150,), solver=lbfgs; total time=  49.4s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END alpha=0.01, hidden_layer_sizes=(150,), solver=lbfgs; total time=  34.5s
Mejor configuración encontrada:
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'adam'}
Mejor F1-macro: 0.8994

Resultados detallados por configuración:
{'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'adam'} -> F1-macro: 0.8857 (+/-0.0084)
{'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'lbfgs'} -> F1-macro: 0.8470 (+/-0.0136)
{'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'solver': 'adam'} -> F1-macro: 0.8927 (+/-0.0163)
{'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'solver': 'lbfgs'} -> F1-macro: 0.8517 (+/-0.0096)
{'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'solver': 'adam'} -> F1-macro: 0.8880 (+/-0.0141)
{'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'solver': 'lbfgs'} -> F1-macro: 0.8590 (+/-0.0159)
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'adam'} -> F1-macro: 0.8994 (+/-0.0080)
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'lbfg

### Usando los mejores valores de gridsearch

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(150,), max_iter=2000, alpha=0.0001, solver='lbfgs', random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
mlp_scores = cross_val_score(mlp_model, X_train_reduced, y_resampled, cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Multi-layer Perceptron (MLPClassifier):")
print(f"F1-macro scores por fold: {mlp_scores}")
print(f"F1-macro promedio: {mlp_scores.mean():.4f}")

Multi-layer Perceptron (MLPClassifier):
F1-macro scores por fold: [0.83650344 0.8066538  0.9079444  0.88073608 0.89790197]
F1-macro promedio: 0.8659


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

# Entrenar el modelo en todo el conjunto de entrenamiento
mlp_model.fit(X_train_reduced, y_resampled)

# Predecir sobre el conjunto de validación/desarrollo
y_pred = mlp_model.predict(X_dev_reduced)
y_true = y_dev

# Imprimir el reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_true, y_pred, digits=4))

# Imprimir la matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_true, y_pred))

Reporte de clasificación:
              precision    recall  f1-score   support

           0     0.7890    0.9182    0.8487       501
           1     0.6496    0.3819    0.4810       199

    accuracy                         0.7657       700
   macro avg     0.7193    0.6500    0.6649       700
weighted avg     0.7494    0.7657    0.7442       700

Matriz de confusión:
[[460  41]
 [123  76]]
